<a href="https://colab.research.google.com/github/DockingBlade/Communities-and-Crime/blob/main/Communities_and_Crime_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing as sklpp
from sklearn import decomposition as skldecomp 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LassoLarsCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error

In [ ]:

df = pd.read_csv('communities.csv');
numDataSamples = df.shape[0];
numAttributes = df.shape[1];
print('Number of Attributes before Processing, ', numAttributes, '\n');
df = df.drop(['state'],axis =1 )
df = df.drop(['county'],axis =1 )
df = df.drop(['community'],axis =1 )
df = df.drop(['communityname'],axis =1 )
df = df.drop(['fold'],axis =1 )
df = df.drop(['OtherPerCap'],axis =1 )
df = df.drop(['LemasSwornFT'],axis =1 )
df = df.drop(['LemasSwFTPerPop'],axis =1 )
df = df.drop(['LemasSwFTFieldOps'],axis =1 )
df = df.drop(['LemasSwFTFieldPerPop'],axis =1 )
df = df.drop(['LemasTotalReq'],axis =1 )
df = df.drop(['LemasTotReqPerPop'],axis =1 )
df = df.drop(['PolicReqPerOffic'],axis =1 )
df = df.drop(['PolicPerPop'],axis =1 )
df = df.drop(['RacialMatchCommPol'],axis =1 )
df = df.drop(['PctPolicWhite'],axis =1 )
df = df.drop(['PctPolicBlack'],axis =1 )
df = df.drop(['PctPolicHisp'],axis =1 )
df = df.drop(['PctPolicAsian'],axis =1 )
df = df.drop(['PctPolicMinor'],axis =1 )
df = df.drop(['OfficAssgnDrugUnits'],axis =1 )
df = df.drop(['NumKindsDrugsSeiz'],axis =1 )
df = df.drop(['PolicAveOTWorked'],axis =1 )
df = df.drop(['PolicCars'],axis =1 )
df = df.drop(['PolicOperBudg'],axis =1 )
df = df.drop(['LemasPctPolicOnPatr'],axis =1 )
df = df.drop(['LemasGangUnitDeploy'],axis =1 )
df = df.drop(['PolicBudgPerPop'],axis =1 )


for index in df.columns:
    df.loc[((df[index] > 1) | (df[index] < 0)), index]= pd.NA

if df.isnull().values.any():
    print('Dataframe contains invalid values ')
else:
    print('All entries in the data are valid.')

numDataSamples = df.shape[0];
numAttributes = df.shape[1];

print(df);

Number of Attributes before Processing,  128 

All entries in the data are valid.
     population householdsize  ... LemasPctOfficDrugUn ViolentCrimesPerPop
0          0.19          0.33  ...                0.32                 0.2
1             0          0.16  ...                   0                0.67
2             0          0.42  ...                   0                0.43
3          0.04          0.77  ...                   0                0.12
4          0.01          0.55  ...                   0                0.03
...         ...           ...  ...                 ...                 ...
1989       0.01           0.4  ...                   0                0.09
1990       0.05          0.96  ...                   0                0.45
1991       0.16          0.37  ...                0.91                0.23
1992       0.08          0.51  ...                0.22                0.19
1993        0.2          0.78  ...                   1                0.48

[1994 rows x 100 

This dataset is a list of population parameters of different communities around the country, and the goal is to predict the number of violent crimes committed in the communities per capita. All the data has been normalize to the range 0.0 to 1.0, in a way that maintains the distribution of the data. Note the normalization has perserved the ratios within the attribute, except for values above 3 Standred Deviation which have been set to 1, and values below 3 Standred Deviation which have been set to 0. However ratios between different attributes have not been maintained. Finally, only communities with more than a 100 police officers have taken the LEMAS survey, because of this a lot of communities don't have data for many attributes, to be specific 1675 of them. Thus I decided to drop these attributes. Furthermore I have dropped the state, county, community, community name, and fold becuase the dataset labeled these as non-predictive. After this the number of attributes dropped from 128 to 100, with 99 independent variables, and 1 dependent variable. The dataset contains 1994 samples. All the variables are numeric, except community name, which was dropped. 

In [ ]:
preX = df.iloc[:,np.r_[0:numAttributes-1]].to_numpy();
labels =  df.loc[:,'ViolentCrimesPerPop'].to_numpy();

mean_datascaler = sklpp.StandardScaler(with_mean=True, with_std=False);
data_pca = skldecomp.PCA(n_components=0.95, svd_solver='full');



X_train, X_test, y_train, y_test = train_test_split(preX, labels, test_size=0.20, shuffle=True);

X_train = mean_datascaler.fit_transform(X_train);
X_train =  data_pca.fit_transform(X_train);
mean = np.array(mean_datascaler.mean_);

test_X = X_test.astype(np.float64, copy=False)
for i in range(np.shape(test_X)[0]):
  test_X[i,:] -= mean;

U = np.transpose(data_pca.components_);
U_transpose = np.transpose(U);
Xtest_transpose = U_transpose @ np.transpose(test_X);
X_test = np.transpose(Xtest_transpose);

print(X_train.shape);


(1595, 31)


Here I do PCA, to reduce the dimensonality of the data. I set the minimum amount of energy to capture to 95%, and it reduced the number of attributes from 99 to 31. I also split the data into training and test data, with 80% into training, and 20% test. I did PCA on the training data, extracted the mean, and then I took subtracted the mean from each row in the test data, and used the components calculated by PCA to project the test data to the lower dimension.


In [ ]:
def get_score (model, train_input, train_label, test_input, test_label):
  
  model.fit(train_input,np.ravel(train_label));
  y_pred = model.predict(test_input);
  labels = np.ravel(test_label);
  
  return mean_squared_error(labels,np.transpose(y_pred));

In [ ]:
lasso = LassoCV(cv=8).fit(X_train, y_train)
print('The alpha values tested on LASSO with Coordinate Descent  \n', lasso.alphas_,'\n');
print('The value of alpha choosen by LASSO with Coordinate Descent \n', lasso.alpha_, '\n');


print('The R^2 value for LASSO with Coordinate Descent, \n', lasso.score(X_test,y_test));
y_pred = lasso.predict(X_test);
print('The Mean Squared Error for LASSO with Coordinate Descent is ', mean_squared_error(y_test,np.transpose(y_pred)));

The alpha values tested on LASSO with Coordinate Descent  
 [0.13191858 0.12302771 0.11473605 0.10700322 0.09979156 0.09306595
 0.08679361 0.08094401 0.07548866 0.07040098 0.06565619 0.06123118
 0.0571044  0.05325576 0.0496665  0.04631914 0.04319739 0.04028603
 0.03757088 0.03503873 0.03267724 0.0304749  0.028421   0.02650552
 0.02471913 0.02305315 0.02149944 0.02005045 0.01869912 0.01743886
 0.01626354 0.01516743 0.0141452  0.01319186 0.01230277 0.0114736
 0.01070032 0.00997916 0.00930659 0.00867936 0.0080944  0.00754887
 0.0070401  0.00656562 0.00612312 0.00571044 0.00532558 0.00496665
 0.00463191 0.00431974 0.0040286  0.00375709 0.00350387 0.00326772
 0.00304749 0.0028421  0.00265055 0.00247191 0.00230531 0.00214994
 0.00200505 0.00186991 0.00174389 0.00162635 0.00151674 0.00141452
 0.00131919 0.00123028 0.00114736 0.00107003 0.00099792 0.00093066
 0.00086794 0.00080944 0.00075489 0.00070401 0.00065656 0.00061231
 0.00057104 0.00053256 0.00049666 0.00046319 0.00043197 0.00040286
 0.

In [ ]:
lassoLars = LassoLarsCV(cv=8).fit(X_train, y_train)
print('The alpha values tested by LASSO with Least Angle Regression \n', lassoLars.alphas_,'\n');
print('The value of alpha choosen by LASSO with Least Angle Regression \n', lassoLars.alpha_, '\n');


print('The R^2 value for LASSO with Least Angle Regression, \n', lassoLars.score(X_test,y_test));
y_pred = lassoLars.predict(X_test);
print('The Mean Squared Error for LASSO with Least Angle Regression is ', mean_squared_error(y_test,np.transpose(y_pred)));

The alpha values tested by LASSO with Least Angle Regression 
 [3.23164857e-03 2.06935151e-03 1.43739495e-03 1.36934800e-03
 1.20738413e-03 9.44412504e-04 3.91558891e-04 3.75037430e-04
 2.73010879e-04 2.64388474e-04 2.56299460e-04 2.14973891e-04
 1.73580224e-04 1.67957139e-04 1.54736550e-04 1.48733519e-04
 1.45604496e-04 1.44065858e-04 1.39513908e-04 1.15555615e-04
 9.27363677e-05 8.88513916e-05 8.83179670e-05 8.23377243e-05
 7.15973473e-05 6.62511729e-05 6.50242387e-05 5.45964508e-05
 4.15514284e-05 2.18847936e-05 1.59841259e-05] 

The value of alpha choosen by LASSO with Least Angle Regression 
 1.5984125943728795e-05 

The R^2 value for LASSO with Least Angle Regression, 
 0.6659488539500655
The Mean Squared Error for LASSO with Least Angle Regression is  0.01941143908740388


First Model is LASSO. There are two versions, one optimized with the Least Angle Regression Method, and the other optimized with coordinate descent. These implementations, already have Cross Validation built into them to determine the best value of $\lambda$ or in the sklearn documentation $\alpha$. The implementations, have 100 precomputed alphas it has chosen to train on, and I decided to use them. 

In [ ]:
empty = [];
parameters = np.array(empty);

lam = 0.00001;

for i in range(0,10):
  parameter = lam; 
  for j in range(0,9):
    parameters = np.append(parameters,parameter);
    parameter += lam;
  lam *= 10;

  parameters = np.append(parameters,[11,12,13,14,15,16,17,18,19,20]);

ridge = RidgeCV(store_cv_values=True, alphas=parameters).fit(X_train,y_train);
print('The alpha value choosen by RidgeCV is \n',ridge.alpha_)
print('The values that alpha was choosen from for the RidgeCV \n', ridge.alphas)

print('The R^2 value for Ridge Regression is, \n', ridge.score(X_test,y_test));
y_pred = ridge.predict(X_test);
print('The Mean Squared Error for LASSO with LARS is ', mean_squared_error(y_test,np.transpose(y_pred)));


The alpha value choosen by RidgeCV is 
 13.0
The values that alpha was choosen from for the RidgeCV 
 [1.0e-05 2.0e-05 3.0e-05 4.0e-05 5.0e-05 6.0e-05 7.0e-05 8.0e-05 9.0e-05
 1.1e+01 1.2e+01 1.3e+01 1.4e+01 1.5e+01 1.6e+01 1.7e+01 1.8e+01 1.9e+01
 2.0e+01 1.0e-04 2.0e-04 3.0e-04 4.0e-04 5.0e-04 6.0e-04 7.0e-04 8.0e-04
 9.0e-04 1.1e+01 1.2e+01 1.3e+01 1.4e+01 1.5e+01 1.6e+01 1.7e+01 1.8e+01
 1.9e+01 2.0e+01 1.0e-03 2.0e-03 3.0e-03 4.0e-03 5.0e-03 6.0e-03 7.0e-03
 8.0e-03 9.0e-03 1.1e+01 1.2e+01 1.3e+01 1.4e+01 1.5e+01 1.6e+01 1.7e+01
 1.8e+01 1.9e+01 2.0e+01 1.0e-02 2.0e-02 3.0e-02 4.0e-02 5.0e-02 6.0e-02
 7.0e-02 8.0e-02 9.0e-02 1.1e+01 1.2e+01 1.3e+01 1.4e+01 1.5e+01 1.6e+01
 1.7e+01 1.8e+01 1.9e+01 2.0e+01 1.0e-01 2.0e-01 3.0e-01 4.0e-01 5.0e-01
 6.0e-01 7.0e-01 8.0e-01 9.0e-01 1.1e+01 1.2e+01 1.3e+01 1.4e+01 1.5e+01
 1.6e+01 1.7e+01 1.8e+01 1.9e+01 2.0e+01 1.0e+00 2.0e+00 3.0e+00 4.0e+00
 5.0e+00 6.0e+00 7.0e+00 8.0e+00 9.0e+00 1.1e+01 1.2e+01 1.3e+01 1.4e+01
 1.5e+01 1.6e+01 1.7e+

Second Model is Ridge Regression. Ridge Regression has a closed from solution, thus there is no optimizer. This implementations, already have Cross Validation built into them to determine the best value of $\lambda$ or in the sklearn documentation $\alpha$. This implementation didn't have a large list of alphas to test, so I create my own array of alphas to test with order of magnitude ranging from $10^{-5}$ to $10^{4}$. 

In [ ]:
ratios = [];
ratios = np.array(ratios)

ratio = 0.01;
for index in range(0,100):
  ratios = np.append(ratios,ratio);
  ratio+= 0.01;

elasticNet = ElasticNetCV(l1_ratio=ratios).fit(X_train,y_train);

print('The alpha value choosen by ElasticNetCV is \n',elasticNet.alpha_)
print('The values that alpha was choosen from for ElasticNetCV \n', elasticNet.alphas_)

print('The R^2 value for ElasticNet Regression is, \n', elasticNet.score(X_test,y_test));
y_pred = elasticNet.predict(X_test);
print('The Mean Squared Error for LASSO with LARS is ', mean_squared_error(y_test,np.transpose(y_pred)));


The alpha value choosen by ElasticNetCV is 
 0.006595928795375019
The values that alpha was choosen from for ElasticNetCV 
 [[1.31918576e+01 1.23027705e+01 1.14736050e+01 ... 1.51674305e-02
  1.41451965e-02 1.31918576e-02]
 [6.59592880e+00 6.15138527e+00 5.73680249e+00 ... 7.58371527e-03
  7.07259825e-03 6.59592880e-03]
 [4.39728586e+00 4.10092351e+00 3.82453499e+00 ... 5.05581018e-03
  4.71506550e-03 4.39728586e-03]
 ...
 [1.34610792e-01 1.25538475e-01 1.17077602e-01 ... 1.54769699e-04
  1.44338740e-04 1.34610792e-04]
 [1.33251087e-01 1.24270410e-01 1.15895000e-01 ... 1.53206369e-04
  1.42880773e-04 1.33251087e-04]
 [1.31918576e-01 1.23027705e-01 1.14736050e-01 ... 1.51674305e-04
  1.41451965e-04 1.31918576e-04]]
The R^2 value for ElasticNet Regression is, 
 0.6667404691961076
The Mean Squared Error for LASSO with LARS is  0.01936543897241876


The last Model is ElasticNet. This implementation is optimized with coordinate descent, and has Cross Validation built into them to determine the best value of $\lambda$ or in the sklearn documentation $\alpha$. The implementations, have 100 precomputed alphas it has chosen to train on, and I decided to use them. The alpha is a little different here. The model we learned in class for elastic net is 
<br>
$w$ = $\displaystyle\min_{w} \alpha_2 * ||w||_2^{2} + \alpha_1 * ||w||_1 +  \sum_{i=1}^{n} ||y - Xw||_2^{2}$

<br> 
sklearn implements it as follows:
<br> 
$w$ = $\displaystyle\min_{w}  \frac{\alpha(1-\rho)}{2} ||w||_2^{2} + \alpha \rho ||w||_1 +  \frac{1}{2n}\sum_{i=1}^{n} ||y - Xw||_2^{2}$
<br>

where $\rho$ is the the $L_1$ ratio, the idea is we pick a bugdet in alpha, and choose how much of that budget goes to the $L_1$ via $\rho$. So I constructed a list of $L_1$ ratios, and the implement takes everything in the catesian product of the set of alphas and $L_1$ ratio and determines which is the best combination. 


The $R^2$ value for LASSO with Coordinate Descent, 0.6661518641273685
<br>
The Mean Squared Error for LASSO with Coordinate Descent is  0.019399642331914678
<br>
<br>
The $R^2$ value for LASSO with Least Angle Regression, 0.6659488539500655
<br>
The Mean Squared Error for LASSO with Least Angle Regression is  0.01941143908740388
<br>
<br>
The $R^2$ value for Ridge Regression is 0.666044552359101
<br>
The Mean Squared Error for LASSO with LARS is  0.01940587813106615
<br>

The R^2 value for ElasticNet Regression is, 0.6667404691961076 <br>
The Mean Squared Error for LASSO with LARS is  0.01936543897241876<br>

Let p be the number of features, n be the number of samples, and e be the number of iterations

The Time Complexity of training LASSO with LARS is: <br>
 $O(np^2)$ 
 <br>
The Time Complexity of training LASSO with coordinate descent is: <br>
$O(npe)$
<br>
The Time Complexity of training Ridge Regression is: <br>
$O(np^2)$
<br>

The Time Complexity of training Elasticnet with coordinate descent is: <br>
$O(npe)$
<br>

Since all of the methods train the same model, and same parameters the time complexity for prediction is <br>
$O(p)$

In the implementation of Lasso, and Elasticnet e is at max 1000, which is greater than p in this case. so we will assume $O(p)$ < $O(e)$

$R^2$ explains the proportion of variation that the independent variables can explain in the dependent variables via the model used. 
<br>
Mean Squared Error is a measure of the error of the model. 

The $R^2$ values are within $10^{-3}$ within each other and MSE values are around $10^{-4}$. So there is very little difference between each model in terms of performance, but Elasticnet has the highest $R^2$ and lowest MSE, and thus is the best performing model. However if training time was a concern, then Ridge Regression is the best choice because it is tied with LASSO with LARS for best time complexity, but has a higher $R^2$ value and lower MSE. However the training was quick, so Elasticnet is an appropiate choice. However the $R^2$ value is around 0.66 which indicates some linear relationship between the independent variables and dependent values, but a non-linear model might be able to better fit the independent variables to the prediction of violent crimes. 


Regarding Ethics, this data set can be used to help predict potentially troubled communities, and allow the authorities to response appropiately. However this presents a problem becauses the authorities might not know how to appropriately react to a predictition of high crime. They may over react and aggrevate the communitiy to a position that is even worse then before. Also, someone might try to use the data to blame minority populations for crime, and justify racism. 


Bibliography: <br>
https://www.stat.cmu.edu/~ryantibs/convexopt-F18/lectures/coord-desc.pdf <br>
https://stats.stackexchange.com/questions/76518/what-is-the-time-complexity-of-lasso-regression <br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html<br>
https://en.wikipedia.org/wiki/Computational_complexity_of_mathematical_operations#Matrix_algebra<br>
https://scikit-learn.org/stable/modules/linear_model.html#elastic-net<br>
https://scikit-learn.org/stable/modules/linear_model.html#least-angle-regression<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsCV.html#sklearn.linear_model.LassoLarsCV
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html<br>